In [148]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from helpers.helper_functions import translate_to_file_string
from pyspark.sql.functions import col,lit,to_date
from pyspark.sql.functions import expr,when
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer
from sklearn.impute import KNNImputer
from pyspark.sql.functions import rand
import random

# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

inputFile = translate_to_file_string("./data/RKI_COVID19_20210529.csv")

### Create Spark Session

In [149]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("RKICOVID19PREPARATION")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- ObjectId: integer (nullable = true)
 |-- IdBundesland: integer (nullable = true)
 |-- Bundesland: string (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- AnzahlFall: integer (nullable = true)
 |-- AnzahlTodesfall: integer (nullable = true)
 |-- Meldedatum: string (nullable = true)
 |-- IdLandkreis: integer (nullable = true)
 |-- Datenstand: string (nullable = true)
 |-- NeuerFall: integer (nullable = true)
 |-- NeuerTodesfall: integer (nullable = true)
 |-- Refdatum: string (nullable = true)
 |-- NeuGenesen: integer (nullable = true)
 |-- AnzahlGenesen: integer (nullable = true)
 |-- IstErkrankungsbeginn: integer (nullable = true)
 |-- Altersgruppe2: string (nullable = true)

None


In [150]:
df.count()

2003106

# Data Preperation

## Datenreinigung
Nimmt das Feld NeuerFall den Wert -1 an, so ist er laut RKI "[...] nur in der Publikation des Vortags enthalten". Das heißt es handelt sich folglich um eine Korrektur, der Puplikation des Vortages und muss in der aktuellen Datenauswertung näher betrachtet werden. Daraus ergibt sich, dass die entsprechenden Records aus dem Dataframe herausgefiltert werden müssen.

In [151]:
df = df.filter(df.NeuerFall > -1)

In [152]:
df.count()

2002544

## Datentransformation
In der Spalte AnzahlFall steht jeweils die Summe der Fälle. Um nun die verschiedenen Modelle trainieren zu können, muss nun die Aggration rückgängig gemacht werden. Das heißt für jeden Fall muss nun ein Record im DataFrame aufgenommen werden. Die neue Anzahl der Records muss der Anzahl der gemeldeten Fälle entsprechen. Darüber hinaus ist eine neue Spalte anzufügen. Die neue Spalte gibt an ob die Person genesen, gestorben oder keins von beiden ist. Die Spalten AnzahlFall, AnzahlTodesfall und AnzahlGenesen können dann entfallen. Ebenfalls die Felder NeuerFall, NeuerTodesfall und NeuGenesen. 

### Vereinzelung

In [153]:
df = df.withColumn("AnzahlFall", expr("explode(array_repeat(AnzahlFall,int(AnzahlFall)))"))
df.count()

3675296

### Neue Spalte

In [154]:
df = df.withColumn("FallStatus", when(df.AnzahlGenesen > 0, "GENESEN")
                                 .when(df.AnzahlTodesfall > 0, "GESTORBEN")
                                 .otherwise("NICHTEINGETRETEN"))

In [155]:
df.show()

+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|AnzahlFall|AnzahlTodesfall|          Meldedatum|IdLandkreis|          Datenstand|NeuerFall|NeuerTodesfall|            Refdatum|NeuGenesen|AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|FallStatus|
+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|         3|              0|2021/03/19 00:00:...|       1001|29.05.2021, 00:

## Datenreduktion
In dem folgenden Schritt werden die nicht notwendigen Spalten gelöscht. Spalte Altersgruppe2 ist nicht mit konkreten Werten befüllt und kann daher entfernt werden. Die Informationen aus AnzahlTodesfall und AnzahlGenesen bzw. NeuerTodefall und NeuGenesen sind durch das neue Feld FallStatus abgebildet. Die Felder AnzahlFall und NeuerFall sind durch die Vereinzelung und das Herausfiltern von Korrekturwerten überflüssig geworden. Die ObjectId hat an dieser Stelle auch keine Aussagekraft, da nicht mehrere Puplikationen verglichen werden, sonderns jeweils nur die Aktuelle Puplikation betrachtet wird. Ebenfalls nicht notwendig ist das Feld "IstErkrankungsbeginn".

In [156]:
# definition der zu löschenden Spalten
columnsToDelete = ("Altersgruppe2", "AnzahlFall", "NeuerFall", "AnzahlTodesfall", "NeuerTodesfall", "AnzahlGenesen", "NeuGenesen", "IstErkrankungsbeginn", "ObjectId")
df = df.drop(*columnsToDelete)

In [157]:
df.show()

+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+--------------------+----------+
|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|          Datenstand|            Refdatum|FallStatus|
+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+--------------------+----------+
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34

## Imputation fehlender Werte
Fehlende Werte treten häufig in Datensätzen auf. Dies kann zu Problemen während dem Modelling führen. Aus diesem Grund gibt es verschiedene Möglichkeiten damit umzugehen. Das einfache Löschen von Datensätzen mit fehlenden Werten kann zu einer Verzerrung führen. Daher werden of Machine Learing Techniken angewandt um plausible Werte für die einzelen Features zu finden. (García et al. 2016, 4) In dem vorliegenden Datensatz des RKIs ist in bestimmten Fällen das Geschlecht unbekannt. Da der der Imputer von PySpark jedoch nicht für kategorische Werte geignet ist (Apache Spark 2021), wurde an dieser Stelle, das Verhältnis zwischen dem Männlichen und Weiblichen Geschlecht ermittelt und per Zufallswert das Geschlecht vergeben.

In [158]:
countWoman = df.filter(df.Geschlecht == "W").count()
countMan = df.filter(df.Geschlecht == "M").count()
countAll = countWoman + countMan
print("Anzahl Frauen: ", countWoman)
print("Anzahl Männer: ", countMan)

Anzahl Frauen:  1880476
Anzahl Männer:  1769786


In [163]:
df = df.withColumn("random", (rand() * countAll))
df = df.withColumn("randomGender", when(df.random > countWoman, "M").otherwise("W"))
df = df.withColumn("Geschlecht", when(df.Geschlecht == "unbekannt", df.randomGender).otherwise(df.Geschlecht)).drop("random","randomGender")
df.filter(df.Geschlecht == "unbekannt").count()

0